## Generate data from GPX

In [10]:
import gpxpy
import gpxpy.gpx
from geopy.distance import geodesic
import statistics
import gpxpy
import gpxpy.gpx
from geopy.distance import geodesic
from geopy import distance
import json



race_name = "Boston Marathon"

file_path = f"gpx_race/{race_name}.gpx"

## Routine

In [11]:


def analyze_gpx_point_to_point(gpx_file):
    with open(gpx_file, 'r') as file:
        gpx = gpxpy.parse(file)

    total_length = 0
    segments_lengths = []

    # Calculer la distance entre chaque point successif
    for track in gpx.tracks:
        for segment in track.segments:
            for i in range(len(segment.points) - 1):
                point_a = segment.points[i]
                point_b = segment.points[i+1]
                segment_length = geodesic(
                    (point_a.latitude, point_a.longitude),
                    (point_b.latitude, point_b.longitude)).meters
                segments_lengths.append(segment_length)
                total_length += segment_length

    # Statistiques des segments
    longest_segment = max(segments_lengths) if segments_lengths else 0
    shortest_segment = min(segments_lengths) if segments_lengths else 0
    average_length = statistics.mean(segments_lengths) if segments_lengths else 0
    num_segments = len(segments_lengths)

    return {
        'total_length': total_length,
        'number_of_segments': num_segments,
        'longest_segment': longest_segment,
        'shortest_segment': shortest_segment,
        'average_segment_length': average_length
    }

# Utilisation de la fonction
gpx_stats = analyze_gpx_point_to_point(file_path)
print("Total Length of Course: {:.2f} meters".format(gpx_stats['total_length']))
print("Number of Segments: {}".format(gpx_stats['number_of_segments']))
print("Longest Segment: {:.2f} meters".format(gpx_stats['longest_segment']))
print("Shortest Segment: {:.2f} meters".format(gpx_stats['shortest_segment']))
print("Average Segment Length: {:.2f} meters".format(gpx_stats['average_segment_length']))


Total Length of Course: 42521.19 meters
Number of Segments: 3886
Longest Segment: 16.71 meters
Shortest Segment: 3.25 meters
Average Segment Length: 10.94 meters


## Créer le json des directions

In [22]:
import gpxpy
import gpxpy.gpx
from geopy.distance import geodesic
from geographiclib.geodesic import Geodesic
import json

def calculate_directions(gpx_file):
    # Charger le fichier GPX
    with open(gpx_file, 'r') as file:
        gpx = gpxpy.parse(file)

    segments_directions = []
    # Itérer sur tous les segments dans toutes les pistes
    for track in gpx.tracks:
        for segment in track.segments:
            i = 0
            while i < len(segment.points) - 1:
                segment_start = segment.points[i]
                total_distance = 0.0
                # Trouver le point à au moins 100 mètres
                for j in range(i + 1, len(segment.points)):
                    step_distance = geodesic(
                        (segment.points[j-1].latitude, segment.points[j-1].longitude),
                        (segment.points[j].latitude, segment.points[j].longitude)).meters
                    total_distance += step_distance
                    if total_distance >= 100:
                        break
                if j < len(segment.points):
                    # Calculer la direction
                    initial_pos = (segment_start.latitude, segment_start.longitude)
                    final_pos = (segment.points[j].latitude, segment.points[j].longitude)
                    azimuth = Geodesic.WGS84.Inverse(initial_pos[0], initial_pos[1], final_pos[0], final_pos[1])['azi1']
                    segments_directions.append({'start': initial_pos, 'end': final_pos, 'direction': azimuth,'length':total_distance})
                    i = j
                else:
                    break

    # Convertir en JSON
    json_data = json.dumps(segments_directions, indent=2)
    # Déterminer le nom de fichier à partir du nom de fichier GPX
    json_file_name = f"directions_race/{race_name}_directions.json"
    # Sauvegarder le fichier JSON
    with open(json_file_name, 'w') as f: f.write(json_data)



    return json_file_name  # retourner le nom du fichier JSON pour confirmation



# Utilisation de la fonction
directions_json = calculate_directions(file_path)
print(directions_json)


directions_race/Boston Marathon_directions.json
